In [81]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [82]:
#Loading Data Set
column_names = ["MPG","Cylinder","Displacement","Horsepower",
               "Weight","Acceleration","Model Year","Origin"]

raw_dataset = pd.read_csv("auto-mpg.csv",names=column_names,
                          na_values="?",comment="\t",sep=",",index_col=False,skiprows=1)

dataset = raw_dataset.copy()

/var/folders/wk/pw3qh6ln1cz814hcq54lrt800000gn/T/ipykernel_59287/1442304926.py:5: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  raw_dataset = pd.read_csv("auto-mpg.csv",names=column_names,


In [83]:
dataset.tail()

,MPG,Cylinder,Displacement,Horsepower,Weight,Acceleration,Model Year,Origin
393,27.0,4,140.0,86.0,2790,15.6,82,1
394,44.0,4,97.0,52.0,2130,24.6,82,2
395,32.0,4,135.0,84.0,2295,11.6,82,1
396,28.0,4,120.0,79.0,2625,18.6,82,1
397,31.0,4,119.0,82.0,2720,19.4,82,1


In [84]:
dataset.isna().sum()

MPG             0
Cylinder        0
Displacement    0
Horsepower      6
Weight          0
Acceleration    0
Model Year      0
Origin          0
dtype: int64

In [85]:
dataset = dataset.dropna()

In [86]:
dataset.isna().sum()

MPG             0
Cylinder        0
Displacement    0
Horsepower      0
Weight          0
Acceleration    0
Model Year      0
Origin          0
dtype: int64

In [87]:
dataset

,MPG,Cylinder,Displacement,Horsepower,Weight,Acceleration,Model Year,Origin
0,18.0,8,307.0,130.0,3504,12.0,70,1
1,15.0,8,350.0,165.0,3693,11.5,70,1
2,18.0,8,318.0,150.0,3436,11.0,70,1
3,16.0,8,304.0,150.0,3433,12.0,70,1
4,17.0,8,302.0,140.0,3449,10.5,70,1
...,...,...,...,...,...,...,...,...
393,27.0,4,140.0,86.0,2790,15.6,82,1
394,44.0,4,97.0,52.0,2130,24.6,82,2
395,32.0,4,135.0,84.0,2295,11.6,82,1
396,28.0,4,120.0,79.0,2625,18.6,82,1


In [88]:
origin = dataset.pop('Origin')
dataset['USA'] = (origin == 1) * 1.0
dataset['Europe'] = (origin == 2) * 1.0
dataset['Japan'] = (origin == 3) * 1.0
dataset.tail()

,MPG,Cylinder,Displacement,Horsepower,Weight,Acceleration,Model Year,USA,Europe,Japan
393,27.0,4,140.0,86.0,2790,15.6,82,1.0,0.0,0.0
394,44.0,4,97.0,52.0,2130,24.6,82,0.0,1.0,0.0
395,32.0,4,135.0,84.0,2295,11.6,82,1.0,0.0,0.0
396,28.0,4,120.0,79.0,2625,18.6,82,1.0,0.0,0.0
397,31.0,4,119.0,82.0,2720,19.4,82,1.0,0.0,0.0


In [89]:
#Train Test Splitting

In [90]:
#80% Data - Training ,20% -Testing
train_dataset = dataset.sample(frac=0.8,random_state=0)

In [91]:
#20% remaining - Testing
test_dataset = dataset.drop(train_dataset.index)

In [92]:
train_labels = train_dataset.pop('MPG')
test_labels = test_dataset.pop('MPG')
#separating labels from the features
#MPG - Miles per Gallon

In [93]:
train_stats=train_dataset.describe().transpose()
train_stats

,count,mean,std,min,25%,50%,75%,max
Cylinder,314.0,5.477707,1.699788,3.0,4.00,4.0,8.00,8.0
Displacement,314.0,195.318471,104.331589,68.0,105.50,151.0,265.75,455.0
Horsepower,314.0,104.869427,38.096214,46.0,76.25,94.5,128.00,225.0
Weight,314.0,2990.251592,843.898596,1649.0,2256.50,2822.5,3608.00,5140.0
Acceleration,314.0,15.559236,2.789230,8.0,13.80,15.5,17.20,24.8
Model Year,314.0,75.898089,3.675642,70.0,73.00,76.0,79.00,82.0
USA,314.0,0.624204,0.485101,0.0,0.00,1.0,1.00,1.0
Europe,314.0,0.178344,0.383413,0.0,0.00,0.0,0.00,1.0
Japan,314.0,0.197452,0.398712,0.0,0.00,0.0,0.00,1.0


In [94]:
#normalization - Feature scaling
def norm(x):
    return (x-train_stats['mean']) / train_stats['std']

normed_train_data = norm(train_dataset)
normed_test_data = norm(test_dataset)

In [95]:
#predictive Model
def build_model():
    model = keras.Sequential(
        # layers of neural networks 64 ,first Hidden layer
        #input => len( no of cols in data set)
        #activation function which we apply on value of each neuron
        #val for each neuron
        [layers.Dense(64,activation="relu",input_shape=[len(train_dataset.keys())]),
        layers.Dense(64,activation="relu"),
        layers.Dense(1)
        ])
    optimizer = tf.keras.optimizers.RMSprop(0.001)
#loss function is used to calculate the difference b/w actual and predicted value
    model.compile(loss="mse",
                 #update weights of neural Network
                 optimizer = optimizer,
                 #mean absolute error
                 metrics = ["mae","mse"])
    return model

In [96]:
model = build_model()
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 64)                640       
                                                                 
 dense_7 (Dense)             (None, 64)                4160      
                                                                 
 dense_8 (Dense)             (None, 1)                 65        
                                                                 
Total params: 4,865
Trainable params: 4,865
Non-trainable params: 0
_________________________________________________________________


In [97]:
epochs = 1000
history = model.fit(normed_train_data,train_labels,epochs = epochs)

Epoch 1/1000
10/10 [==============================] - 0s 606us/step - loss: 571.0823 - mae: 22.5735 - mse: 571.0823
Epoch 2/1000
10/10 [==============================] - 0s 553us/step - loss: 503.4346 - mae: 21.0884 - mse: 503.4346
Epoch 3/1000
10/10 [==============================] - 0s 721us/step - loss: 436.4206 - mae: 19.5203 - mse: 436.4206
Epoch 4/1000
10/10 [==============================] - 0s 569us/step - loss: 363.0433 - mae: 17.6705 - mse: 363.0433
Epoch 5/1000
10/10 [==============================] - 0s 660us/step - loss: 287.9706 - mae: 15.5185 - mse: 287.9706
Epoch 6/1000
10/10 [==============================] - 0s 616us/step - loss: 215.0082 - mae: 13.1724 - mse: 215.0082
Epoch 7/1000
10/10 [==============================] - 0s 601us/step - loss: 148.8970 - mae: 10.7778 - mse: 148.8970
Epoch 8/1000
10/10 [==============================] - 0s 544us/step - loss: 95.1826 - mae: 8.4400 - mse: 95.1826
Epoch 9/1000
10/10 [==============================] - 0s 558us/step - loss:

In [98]:
#model Testing

In [99]:
loss,mae,mse = model.evaluate(normed_test_data,test_labels,verbose=0)

In [54]:
test_predictions = model.predict(normed_test_data).flatten()
test_predictions

NameError: name 'normed_test_data' is not defined

In [55]:
test_labels

9      15.0
25     10.0
28      9.0
31     25.0
33     19.0
       ... 
369    34.0
375    36.0
382    34.0
384    32.0
396    28.0
Name: MPG, Length: 78, dtype: float64

In [101]:
keras_file = "automobile.h5"
tf.keras.models.save_model(model,keras_file)
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tfmodel = converter.convert()
open("linear.tflite","wb").write(tfmodel)

INFO:tensorflow:Assets written to: /var/folders/wk/pw3qh6ln1cz814hcq54lrt800000gn/T/tmp9g_hiyzu/assets


INFO:tensorflow:Assets written to: /var/folders/wk/pw3qh6ln1cz814hcq54lrt800000gn/T/tmp9g_hiyzu/assets
2024-06-26 16:54:22.739876: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2024-06-26 16:54:22.739894: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.


21368